In [1]:
import math
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

### Hooghoudt equation combined with Van der Molen and Wesseling equation.


$R = \frac{8.K_b.D_e.h}{L^2}+\frac{4.K_a.h^2}{L^2}$

Equivalent depth:
$D_e = \frac{\pi.L}{8(ln(\frac{L}{\pi.r_e})+F(x))}$ with $x = \frac{2.\pi.D}{L}$


if x < 0.5 & x > 10$^{-6}$


$F(x)=\frac{\pi^2}{4x}+ln(\frac{x}{2\pi})$

if x >= 0.5

$F(x) = \sum_{n=1}^{\infty}\frac{4exp(-2nx)}{n[1-exp(-2nx)]}$, $n = 1, 3 , 5 ,...$

In [2]:
def Hooghoudt(L_estim, D, h, r_e, R, Ka, Kb):
    
    # Auxiliary variable x (Van der Molen and Wesseling, 1991)
    x = (2*math.pi*D)/L_estim  
    #print(x)
    #print('Auxiliary variable x is ', x)
    
    # Calculate equivalent depth
    D_e = (math.pi*L_estim) / ( 8 * (math.log( (L_estim)/(math.pi*r_e) ) + Vandermolen(x) ) ) 
    #print('D_e is', D_e)
    
    # Calculate L with Hooghoudt equation
    L_calc = math.sqrt( ((8*Kb*D_e*h) + (4*Ka*(h**2))) / R)
    #print('Calculated drain spacing is', L_calc )
    #print(L_estim,L_calc,D,h,r_e,x)
    
    return D_e, L_calc

In [3]:
def Vandermolen(x):
    #Fx = None   # assign a default value of None
    if x > 0.5:
        Fx = 0
        for n in range(1, 1001, 2):
            Fx += 4 * math.exp(-2 * n * x) / (n * (1 - math.exp(-2 * n * x)))
#         print('Series')
        #d = (math.pi * L) / (4 * Fx + math.log(x / (2 * math.pi)))
    else:
        Fx = (((math.pi)**2)/(4*x)) + math.log(x/(2*math.pi)) 
#         print('Analytic')
            #d = (math.pi * L) / (4 * Fx + math.log(x / (2 * math.pi)))
    return Fx

## Case 1

In [ ]:
H = 6.8
Ka = 0.14 #m/d
Kb = Ka
R = 0.001 #m/d
h = 1 #m
D = H-(h+1)
r_e = 0.10 #m
D

## automatic optimization

In [12]:
def obj_fun(LL,D, h, r_e, R, Ka, Kb):
    # Calculate Hooghoudt equation and Vandemolen
    D_e, L_calc = Hooghoudt(LL, D, h, r_e, R, Ka, Kb)
    # Compare estimated and calculated L and minimize difference
    resid = (LL - L_calc)
    OF = resid**2
    
    return OF

In [ ]:
L_est=75
res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb,))
#print(h)
print('The optimal drain spacing is',res.x)

In [ ]:
# Define bounds for L (optional)

bounds = [(0, None)] # L can't be negative

# Define function to be minimized
fun = lambda L: obj_fun(L, D, h, r_e, R, Ka, Kb)

# Minimize objective function
res = minimize(fun, L_est, bounds=bounds)

# Extract optimized L value
L = res.x[0]
L

In [ ]:
#if no package is available
def optimal_drain_spacing(D, h, r_e, R, Ka, Kb):
    L_est = 75
    threshold = 0.01
    
    while True:
        D_e, L_calc = Hooghoudt(L_est, D, h, r_e, R, Ka, Kb)
        diff = abs(L_est - L_calc)
        
        if diff <= threshold:
            break
        
        L_est = L_calc
        
    print('The optimal drain spacing is', L_calc)
optimal_drain_spacing(D, h, r_e, R, Ka, Kb)
    

### manual optimization

In [ ]:
# Visualize relationship equivalent depth and drain spacing for this particular case
D_es=[]
L_calcs = []
LLs = np.arange(1, 100,.2)
L_est=30

fig, ax = plt.subplots(figsize=(6, 4))
for LL in LLs:
    D_e, L_calc= Hooghoudt(LL, D, h, r_e, R, Ka, Kb)
    L_calcs.append(L_calc)
    D_es.append(D_e)
    ax.plot(LL,D_e,'b.')
    ax.set_xlabel('Drain spacing L (m)')
    ax.set_ylabel('Equivalent depth $D_e$ (m)')
    ax.grid('on')
ax.set_xlim(0,100)
# Find the index of the closest value of L to L_est
index = np.argmin(np.abs(LLs - L_est))

# Use linear interpolation to estimate D_e for L=L_est m
L1, L2 = LLs[index], LLs[index+1]
D_e1, D_e2 = D_es[index], D_es[index+1]
D_e_est = D_e1 + (L_est - L1)*(D_e2 - D_e1)/(L2 - L1)

# Print the result
#print('For L='+str(L_est)+'m, D_e is approximately {:.3f}m'.format(D_e_est))    
       
# ax.plot(Lest, D_e_est, 'ro', markersize=10)
# plt.show()
D_e, L_calc= Hooghoudt(L_est, D, h, r_e, R, Ka, Kb)  
#print(D_e, L_calc)
 
ax.plot(L_est, D_e_est, 'bs', markersize=10)    
# ax.plot(L_calc, D_e, 'rs', markersize=10)
ax.text(L_est, D_e_est+.2, 'Initial Guess', ha='center', va='bottom', color='blue', fontsize=14)

tolerance = .1
diff = float('inf')

while diff > tolerance:
    index = np.argmin(np.abs(LLs - L_est))
    
    # Use linear interpolation to estimate D_e for L=L_est m
    L1, L2 = LLs[index], LLs[index+1]
    #print(L1,L2)
    D_e1, D_e2 = D_es[index], D_es[index+1]
    D_e_est = D_e1 + (L_est - L1)*(D_e2 - D_e1)/(L2 - L1)
    
    # Calculate D_e and L_calc for the current value of Lest
    D_e, L_calc = Hooghoudt(L_est, D, h, r_e, R, Ka, Kb)
    
    # Calculate the difference between Lest and L_calc
    diff = abs(L_calc - L_est)
    #print(diff)
    # Update Lest based on the sign of the difference
    if L_calc > L_est:
        L_est += .5
    else:
        L_est -= .5
    #print(L_est)

# Print the final result
print('For L={:.2f}m, D_e is approximately {:.3f}m'.format(L_calc, D_e))
ax.plot(L_calc, D_e, 'ro', markersize=10)
ax.text(L_calc, D_e-.4, 'Solution', ha='center', va='bottom', color='red', fontsize=14)

plt.show()


## case 2

the same information as case 1 except a ditch with the water depth of 2.5m, bottom width of 0.5m and side slope of 1:1
is replaced by drainage pipe. The design water depth in the ditch is 0.5m. What will be the drain spacing?

In [ ]:
# calculate the wet primeter 
u = 0.5+2*math.sqrt(0.5**2+0.5**2)
r_e = u/math.pi
r_e


In [ ]:

res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
#print(h)
print('The optimal drain spacing is',res.x)

## Case 3

In [ ]:
H = 6.8
Ka = 0.06 #m/d
Kb = 0.3  #m/d
R = 0.001 #m/d
h = 1 #m
D = H-(h+1)
r_e = 0.10 #m
L_est=75

In [ ]:
res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
#print(h)
print('The optimal drain spacing is',res.x)

#### Sensitivity of spacing to Ka, Kb etc

In [ ]:
Kas = np.arange(0.01,1.3,.05)
Ls = []
for Kb in Kas:
    res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
    #print(h)
    #print('The optimal drain spacing is',res.x)
    Ls.append(res.x)

## Ernst Equations

In [20]:
def ernstb(Dv, Db, Dr, h, r_e, R, Ka, Kb):
    u=math.pi * r_e
    # Coefficients of the quadratic equation
    a = 1 / (8 * Kb * Db)
    b = math.log(Dr / u) / (math.pi * Kb)
    c = (Dv / Ka) - (h / R)

    # Calculate the roots of the quadratic equation
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        print("No real roots exist.")
    else:
        root1 = (-b + math.sqrt(discriminant)) / (2 * a)
        root2 = (-b - math.sqrt(discriminant)) / (2 * a)
    
    hv = R*(Dv / Ka)
    hh = R*root1**2 /(8 * Kb * Db)
    hr = R*root1*(math.log(Dr / u) / (math.pi * Kb))

    
    return root1, hv,hh,hr

In [21]:
def ernstt(alpha, Dr, Dv, Dt, Db, h, r_e, R, Kt, Kb):
    # Coefficients of the quadratic equation
    a = 1 / (8*((Kb * Db)+(Kt * Dt)))
    #print(a,Dr)
          
    b = math.log((alpha * Dr) / (math.pi * r_e)) / (math.pi * Kt)

    c = (Dv / Kt) - (h / R)
    
    # Calculate the roots of the quadratic equation
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        print("No real roots exist.")
    else:
        root1 = (-b + math.sqrt(discriminant)) / (2 * a)
        root2 = (-b - math.sqrt(discriminant)) / (2 * a)
#         print("L1 =", root1)
#         print("L2 =", root2)
    #vertical, horizontal and radial components of h 
        
    hv = R*(Dv / Kt)
    hh = R*root1**2 / (8*((Kb * Db)+(Kt * Dt)))
    hr = R*root1*(math.log((alpha * Dr) / (math.pi * r_e)) / (math.pi * Kt))

    
    return root1, hv,hh,hr

##### the Geometry factor (a) obtained by the relaxation method (after Van Beers, 1979)


|       |  1  | 2   | 4   | 8   | 16  | 32  |
|-------|-----|-----|-----|-----|-----|-----|
| 1     | 2.0 | 3.0 | 5.0 | 9.0 | 15.0| 30.0|
| 2     | 2.4 | 3.2 | 4.6 | 6.2 | 8.0 | 10.0|
| 3     | 2.6 | 3.3 | 4.5 | 5.5 | 6.8 | 8.0 |
| 5     | 2.8 | 3.5 | 4.4 | 4.8 | 5.6 | 6.2 |
| 10    | 3.2 | 3.6 | 4.2 | 4.5 | 4.8 | 5.0 |
| 20    | 3.6 | 3.7 | 4.0 | 4.2 | 4.4 | 4.6 |
| 50    | 3.8 | 4.0 | 4.0 | 4.0 | 4.2 | 4.6 |


In [17]:
import numpy as np

# Define the values from the table
K_values = np.array([1, 2, 3, 5, 10, 20, 50])
D_values = np.array([1, 2, 4, 8, 16, 32])
T_values = np.array([    [2.0, 3.0, 5.0, 9.0, 15.0, 30.0],
    [2.4, 3.2, 4.6, 6.2, 8.0, 10.0],
    [2.6, 3.3, 4.5, 5.5, 6.8, 8.0],
    [2.8, 3.5, 4.4, 4.8, 5.6, 6.2],
    [3.2, 3.6, 4.2, 4.5, 4.8, 5.0],
    [3.6, 3.7, 4.0, 4.2, 4.4, 4.6],
    [3.8, 4.0, 4.0, 4.0, 4.2, 4.6]
])

def estimate_T(Db_Dt, Kb_Kt):
    # Find the indices of the two closest values of Db_Dt and Kb_Kt in the table
    Db_Dt_index = np.argmin(np.abs(Db_Dt - D_values))
    Kb_Kt_index = np.argmin(np.abs(Kb_Kt - K_values))

    # Use linear interpolation to estimate T
    T1 = T_values[Kb_Kt_index, Db_Dt_index]
    T2 = T_values[Kb_Kt_index, Db_Dt_index+1]
    T3 = T_values[Kb_Kt_index+1, Db_Dt_index]
    T4 = T_values[Kb_Kt_index+1, Db_Dt_index+1]
    
    T = (
        T1 * (D_values[Db_Dt_index+1] - Db_Dt) * (K_values[Kb_Kt_index+1] - Kb_Kt) +
        T2 * (Db_Dt - D_values[Db_Dt_index]) * (K_values[Kb_Kt_index+1] - Kb_Kt) +
        T3 * (D_values[Db_Dt_index+1] - Db_Dt) * (Kb_Kt - K_values[Kb_Kt_index]) +
        T4 * (Db_Dt - D_values[Db_Dt_index]) * (Kb_Kt - K_values[Kb_Kt_index])
    ) / ((D_values[Db_Dt_index+1] - D_values[Db_Dt_index]) * (K_values[Kb_Kt_index+1] - K_values[Kb_Kt_index]))

    return T

# Example usage
Db_Dt = 4
Kb_Kt = 4
T = estimate_T(Db_Dt, Kb_Kt)
print('For Db/Dt={}, Kb/Kt={}, T={}'.format(Db_Dt, Kb_Kt, T))


For Db/Dt=4, Kb/Kt=4, T=4.45


## Case 4


In [23]:
h=0.7
R = 0.007
Dr = 1
Dt = Dr+.5*h
Ka = 0.5
Kb = 2
Db = 4
alpha = estimate_T(Db/Da, Kb/Ka)

Dv = h
r_e = .05
ernstt(alpha, Dr, Dv, Dt, Db, h, r_e, R, Ka, Kb)
 

(38.01779446609499, 0.0098, 0.14578485407008215, 0.5444151459299176)

## Comparison of Hooghhoudt vs Ernst

In [6]:
R = 0.008 #m/d
Ka = 0.4
D = 4
h = 0.4
r_e = 0.1
Kb = Ka

In [16]:
L_est=15
D_e, L_calc= Hooghoudt(L_est, D, h, r_e, R, Ka, Kb)  
res = optimize.minimize(obj_fun, x0=L_est, args=(D, h, r_e, R, Ka, Kb))
print('The optimal drain spacing by Hooghhoudt is',res.x)

h = 0.4
Dr =4 #assumed
Dv = h
Db=Dr


root1,_,_,_ = ernstb(Dv, Db, Dr, h, r_e, R, Ka, Kb)
print('The optimal drain spacing by Ernstb',root1)


alpha  = 1 #homogenous
h =0.4
Dr = 0 #assumed
Dv = h
Db=4
Da=4.8-Db
root1,_,_,_ = ernstt(alpha, Dr, Dv, Da, Db, h, r_e, R, Ka, Kb)
print('The optimal drain spacing by Ernstt is',root1)


The optimal drain spacing by Hooghhoudt is [16.86658598]
The optimal drain spacing by Ernstb 15.240097539613595
The optimal drain spacing by Ernstt is 15.98544297670495
